In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
!pip install sweetviz
import sweetviz as sv

Mounted at /content/drive
     |████████████████████████████████| 15.1MB 10.5MB/s 
     |████████████████████████████████| 81kB 5.5MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [2]:
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
train_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_err2.csv')
train_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_qui2.csv')
train_prob = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_prob2.csv')

test_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_err2.csv')
test_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_qui2.csv')

In [4]:
train_qui2 = train_qui.copy()
test_qui2 = test_qui.copy()
train_err2 = train_err.copy()
test_err2 = test_err.copy()

# 퀼리티 3,4는 값이 1밖에 없어서 의미가 없기 때문에 제외
# 펌웨어 버전은 이미 err 자료에 있기 때문에 중복으로 제외
# 퀼리티 0은 EDA 결과 퀼리티 2와 피어슨 상관계수 0.99를 보이기 때문에 0과 2 중에 하나만 골랐음
# 0의 결측치 Nan은 15%고 2의 결측치는 4%기 때문에 결측치가 적은 2를 남기고 0은 drop 시킨다.

## 퀼리티 6과 7도 0.81로 강한 상관관계를 보이고 있는데, 하나를 제외해야 하나 고민이 됨
train_qui2.drop(['fwver','quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)
test_qui2.drop(['fwver', 'quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)

test_qui2 = test_qui2.replace(',','', regex=True)
train_qui2 = train_qui2.replace(',','', regex=True)
test_qui2.loc[:,'quality_1':'quality_12'] = test_qui2.loc[:,'quality_1':'quality_12'].astype(float)
train_qui2.loc[:,'quality_1':'quality_12'] = train_qui2.loc[:,'quality_1':'quality_12'].astype(float)

train_qui2['time'] = pd.to_datetime(train_qui['time'], format = '%Y-%m-%d %H:%M:%S')
test_qui2['time'] = pd.to_datetime(test_qui['time'], format = '%Y-%m-%d %H:%M:%S')

train_err2['time'] = pd.to_datetime(train_err2['time'], format = '%Y-%m-%d %H:%M:%S')
test_err2['time'] = pd.to_datetime(test_err2['time'], format = '%Y-%m-%d %H:%M:%S')

In [5]:
# 퀼리티 임계치
# [-1, 0, 1] == -1 카테고리 -1, -1~0 =카테고리 0, 그 이상 = 카테고리 1
qualities = {}

qualities['quality_1'] = [-1,0, 1]
qualities['quality_2'] = [-1,0, 1]
qualities['quality_5'] = [-1,0,1,2,4, 10]
qualities['quality_6'] = [-1,0, 1]
qualities['quality_7'] = [0,1,100, 101]
qualities['quality_8'] = [0,1,2]
qualities['quality_9'] = [0,1,2]
qualities['quality_10'] = [0,1,2,3, 4, 5,7, 10 ,14,15] ##??
qualities['quality_11'] = [-1,0,1]
qualities['quality_12'] = [0,1,2]

## 모델 임계치
## 모델 0,1,2,3,4의 비율을 더하면 99.999% 이므로, 모델 5~8은 other model로 통합하는 것이 바람직 해 보임.

In [6]:
def quality_categorize(dataset):
  data = dataset.copy()
  for key in tqdm(qualities):
    intervals = qualities.get(key)
    for i in range(len(intervals)):
      if intervals[i] == intervals[-1]:
        data[key][data[key] >= intervals[i]] =  intervals[i]
      # elif intervals[i+1] - intervals[i] == 1:
      #   continue
      else:
        data[key][(data[key] >= intervals[i]) & (data[key] < intervals[i+1])] = intervals[i]
  return data


In [105]:
prob_list = list(set(train_prob.user_id))

In [93]:
cols = list(train_qui2.iloc[:,1:-2].columns)

In [27]:
for col in cols:
  qui_train = list(train_qui2[col].value_counts().index)
  qui_test = list(test_qui2[col].value_counts().index)
  print(col, '에서 불일치')
  n = 0
  for item in qui_test:    
    if item not in qui_train:
      n +=1
  print('{}개'.format(n))

quality_1 에서 불일치
11개
quality_2 에서 불일치
286개
quality_5 에서 불일치
1860개
quality_6 에서 불일치
42개
quality_7 에서 불일치
447개
quality_8 에서 불일치
10개
quality_9 에서 불일치
225개
quality_10 에서 불일치
1690개
quality_11 에서 불일치
3개


In [97]:
###### 그냥 쓰자니 불일치가 너무 많아서... 전처럼 카테고리화 해서 확률로 전환한다.

train_qui3 = quality_categorize(train_qui2)
test_qui3 = quality_categorize(test_qui2)

In [28]:
# 불일치가 사라졌는지 확인해본다.

for col in cols:
  qui_train = list(train_qui3[col].value_counts().index)
  qui_test = list(test_qui3[col].value_counts().index)
  print(col, '에서 불일치')
  n = 0
  for item in qui_test:    
    if item not in qui_train:
      n +=1
  print('{}개'.format(n))

quality_1 에서 불일치
0개
quality_2 에서 불일치
0개
quality_5 에서 불일치
0개
quality_6 에서 불일치
0개
quality_7 에서 불일치
0개
quality_8 에서 불일치
0개
quality_9 에서 불일치
0개
quality_10 에서 불일치
0개
quality_11 에서 불일치
0개


In [98]:
## 각 퀼리티의 숫자를 % 확률로 변환

train_qui3['problems'] = train_qui3.user_id.isin(prob_list)
train_qui3_tr = train_qui3[train_qui3['problems']].dropna()
train_qui3_fls = train_qui3[train_qui3['problems'] != True].dropna()

In [88]:
def probability(train, test):

  cols = list(train.loc[:,'quality_1':'quality_12'].columns)

  train['problems'] = train.user_id.isin(prob_list)
  train_tr = train[train['problems']]
  train_fls = train[train['problems'] != True]


  for col in tqdm(cols):
    temp_ori = train[col].value_counts()
    temp_tr = train_tr[col].value_counts()
    temp_fls = train_fls[col].value_counts()

    ori = pd.DataFrame(temp_ori)
    tr = pd.DataFrame(temp_tr)

    probs = tr/ori
    for ind in list(probs.index):
      train[col][train[col] == ind] = probs.loc[ind,:].values[0]
      test[col][test[col] == ind] = probs.loc[ind,:].values[0]
  return train, test

In [89]:
train_qui_prob, test_qui_prob = probability(train_qui3, test_qui3)

In [101]:
train_qui3.head(10)

,user_id,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,time,problems
0,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
1,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
2,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
3,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
4,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
5,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
6,10000,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
7,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
8,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
9,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False


In [102]:
train_qui_prob.head(10)

,user_id,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,time,problems
0,10000,0.453202,0.470339,0.393935,0.453063,0.465027,0.463237,0.47047,0.41188,0.453996,0.468568,2020-11-29 09:00:00,False
1,10000,0.453202,0.470339,0.393935,0.453063,0.465027,0.463237,0.47047,0.41188,0.453996,0.468568,2020-11-29 09:00:00,False
2,10000,0.453202,0.470339,0.393935,0.453063,0.465027,0.463237,0.47047,0.41188,0.453996,0.468568,2020-11-29 09:00:00,False
3,10000,0.453202,0.470339,0.393935,0.453063,0.465027,0.463237,0.47047,0.41188,0.453996,0.468568,2020-11-29 09:00:00,False
4,10000,0.453202,0.470339,0.393935,0.453063,0.465027,0.463237,0.47047,0.41188,0.453996,0.468568,2020-11-29 09:00:00,False
5,10000,0.453202,0.470339,0.393935,0.453063,0.465027,0.463237,0.47047,0.41188,0.453996,0.468568,2020-11-29 09:00:00,False
6,10000,0.453202,0.470339,0.542724,0.453063,0.465027,0.463237,0.47047,0.41188,0.453996,0.468568,2020-11-29 09:00:00,False
7,10000,0.453202,0.470339,0.393935,0.453063,0.465027,0.463237,0.47047,0.41188,0.453996,0.468568,2020-11-29 09:00:00,False
8,10000,0.453202,0.470339,0.393935,0.453063,0.465027,0.463237,0.47047,0.41188,0.453996,0.468568,2020-11-29 09:00:00,False
9,10000,0.453202,0.470339,0.393935,0.453063,0.465027,0.463237,0.47047,0.41188,0.453996,0.468568,2020-11-29 09:00:00,False


In [92]:
train_qui_prob.groupby(by = 'user_id').mean()

,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,problems
user_id,,,,,,,,,,,
10000,0.453202,0.470339,0.406334,0.453063,0.465027,0.463237,0.470470,0.420604,0.453996,0.468568,False
10002,0.454698,0.472358,0.405565,0.455809,0.468306,0.463237,0.443576,0.405913,0.455473,0.468568,False
10004,0.459183,0.477247,0.409110,0.460739,0.471586,0.463237,0.470470,0.331158,0.459906,0.468568,True
10005,0.483105,0.504877,0.457266,0.486480,0.471586,0.463237,0.470470,0.420367,0.483546,0.468568,True
10006,0.453202,0.470339,0.406334,0.454166,0.469400,0.463237,0.470470,0.411880,0.453996,0.468568,True
...,...,...,...,...,...,...,...,...,...,...,...
24990,0.477125,0.497970,0.439236,0.477149,0.465027,0.463237,0.470470,0.255056,0.477636,0.468568,True
24992,0.477125,0.497970,0.449984,0.477149,0.465027,0.463237,0.470470,0.419904,0.477636,0.468568,False
24993,0.474134,0.494516,0.441469,0.474138,0.465027,0.463237,0.470470,0.418057,0.474681,0.468568,False


In [104]:
train_qui_prob.groupby(by = 'user_id').sum()

,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,problems
user_id,,,,,,,,,,,
10000,10.876859,11.288142,9.752019,10.873511,11.160652,11.117698,11.291288,10.094486,10.895894,11.245630,0
10002,43.650968,45.346380,38.934247,43.757703,44.957424,44.470793,42.583338,38.967692,43.725417,44.982520,0
10004,11.020392,11.453924,9.818630,11.057741,11.318060,11.117698,11.291288,7.947794,11.037734,11.245630,24
10005,11.594527,12.117051,10.974380,11.675516,11.318060,11.117698,11.291288,10.088801,11.605096,11.245630,24
10006,16.315288,16.932213,14.628028,16.349981,16.898386,16.676547,16.936932,14.827692,16.343841,16.868445,36
...,...,...,...,...,...,...,...,...,...,...,...
24990,11.450993,11.951269,10.541660,11.451572,11.160652,11.117698,11.291288,6.121337,11.463255,11.245630,24
24992,17.176490,17.926903,16.199421,17.177359,16.740978,16.676547,16.936932,15.116535,17.194883,16.868445,0
24993,11.379226,11.868378,10.595247,11.379315,11.160652,11.117698,11.291288,10.033363,11.392335,11.245630,0


In [103]:
test_qui_prob

,user_id,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,time
0,30000,0.453202,0.470339,0.393935,0.453063,0.478145,0.463237,0.47047,0.390104,0.453996,0.468568,2020-11-28 19:50:00
1,30000,0.453202,0.470339,0.393935,0.453063,0.478145,0.463237,0.47047,0.390104,0.453996,0.468568,2020-11-28 19:50:00
2,30000,0.453202,0.470339,0.393935,0.453063,0.478145,0.463237,0.47047,0.390104,0.453996,0.468568,2020-11-28 19:50:00
3,30000,0.453202,0.470339,0.393935,0.453063,0.478145,0.463237,0.47047,0.390104,0.453996,0.468568,2020-11-28 19:50:00
4,30000,0.453202,0.470339,0.393935,0.453063,0.478145,0.463237,0.47047,0.390104,0.453996,0.468568,2020-11-28 19:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...
747967,44997,0.453202,0.470339,0.462466,0.453063,0.465027,0.463237,0.47047,0.407260,0.453996,0.468568,2020-11-25 18:00:00
747968,44997,0.453202,0.470339,0.393935,0.453063,0.465027,0.463237,0.47047,0.407260,0.453996,0.468568,2020-11-25 18:00:00
747969,44997,0.453202,0.470339,0.393935,0.453063,0.465027,0.463237,0.47047,0.407260,0.453996,0.468568,2020-11-25 18:00:00
747970,44997,0.453202,0.470339,0.393935,0.453063,0.465027,0.463237,0.47047,0.407260,0.453996,0.468568,2020-11-25 18:00:00


In [107]:
##### 아니면 단순하게 누적합계를 구하면 어떨까 ########
# 예를 들어 퀼리티 1의 값 0이 3번, 1이 5번, -1이 4번 이렇게 정리하는 방법.

def probability(train, test):

  cols = list(train.loc[:,'quality_1':'quality_12'].columns)

  for col in tqdm(cols):
    temp_ori = train[col].value_counts()
    temp_tr = train_tr[col].value_counts()
    temp_fls = train_fls[col].value_counts()

    ori = pd.DataFrame(temp_ori)
    tr = pd.DataFrame(temp_tr)

    probs = tr/ori
    for ind in list(probs.index):
      train[col][train[col] == ind] = probs.loc[ind,:].values[0]
      test[col][test[col] == ind] = probs.loc[ind,:].values[0]
  return train, test

,user_id,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,time,problems
0,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
1,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
2,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
3,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
4,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,24997,0.0,0.0,1.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00,True
828620,24997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00,True
828621,24997,0.0,0.0,2.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00,True
828622,24997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00,True


In [116]:
values = list(train_qui3.quality_1.value_counts().index)
ids = list(set(train_qui3.user_id))
ids.sort()

In [124]:
def quality_counter(data):
  cols = list(data.loc[:,'quality_1':'quality_12'].columns)
  ids = list(set(data.user_id))
  ids.sort()

  counter = []
  for col in tqdm(cols):
    vals = list(data[col].value_counts().index)
    vals.sort()
    for id in ids:
      temp = data[data.user_id == id]
      val_counter = []
      for val in vals:
        a = len(temp[temp[col] == val])
        val_counter.append(a)
      counter.append([val_counter])
  return counter

In [125]:
tt = quality_counter(train_qui3)

In [143]:
val = []
data = train_qui3
cols = list(data.loc[:,'quality_1':'quality_12'].columns)

temp = data[data.user_id == 10000]
temp[temp['quality_1'] == ]

,user_id,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,time,problems
0,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
1,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
2,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
3,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
4,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
5,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
6,10000,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
7,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
8,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False
9,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00,False


In [142]:
val

[[-1.0, 0.0, 1.0],
 [-1.0, 0.0, 1.0],
 [-1.0, 0.0, 1.0, 2.0, 4.0, 10.0],
 [-1.0, 0.0, 1.0],
 [0.0, 1.0, 100.0, 101.0],
 [0.0, 1.0, 2.0],
 [0.0, 1.0, 2.0],
 [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 7.0, 10.0, 14.0, 15.0],
 [-1.0, 0.0, 1.0],
 [0.0, 1.0, 2.0]]

In [120]:
cols = list(train_qui3.loc[:,'quality_1':'quality_12'].columns)
cols

['quality_1',
 'quality_2',
 'quality_5',
 'quality_6',
 'quality_7',
 'quality_8',
 'quality_9',
 'quality_10',
 'quality_11',
 'quality_12']

In [52]:
probs

,quality_1
0.0,0.448695
-1.0,0.519893
1.0,0.534963


In [ ]:
train_qui2['problems'] = train_qui2.user_id.isin(prob_list)
train_qui2_tr = train_qui2[train_qui2['problems']].dropna()
train_qui2_fls = train_qui2[train_qui2['problems'] != True].dropna()

In [ ]:
## T 검정 진행
from scipy.stats import ttest_ind
from scipy.stats import chisquare
from scipy import stats

In [ ]:
# T 검정을 하기 위해서 두 표본이 등분산인지 확인할 필요가 있음
# 따라서 등분산 검정을 진행
cols = list(train_qui2_tr.iloc[:,1:-2].columns)

r_list = []

for col in cols:
  result = stats.levene(train_qui2_tr[col], train_qui2_fls[col])
  r_list.append(list(result))
#print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))

df = pd.DataFrame(r_list, index = cols, columns = ['통계값', 'P_value'])
df['등분산성'] = df['P_value'].apply(lambda x: '불만족' if x < 0.05 else '만족')
df

,통계값,P_value,등분산성
quality_1,1026.039420,5.488152e-225,불만족
quality_2,5.045943,2.468394e-02,불만족
quality_5,210.015689,1.382207e-47,불만족
quality_6,27.125934,1.906712e-07,불만족
quality_7,51.825960,6.069832e-13,불만족
quality_8,0.244978,6.206340e-01,만족
quality_9,23.230138,1.437538e-06,불만족
quality_10,569.899036,6.548420e-126,불만족
quality_11,3085.176295,0.000000e+00,불만족
quality_12,28.096555,1.154420e-07,불만족


In [ ]:
# quality_8만 등분산성을 만족하고 나머지는 불만족한다..
# ttest_ind에 equal_var=False 을 넣어서 등분산 불만족을 표시해주어야 함.

r_list = []

for col in cols:
  if col == 'quality_8':
    result = ttest_ind(train_qui2_tr[col], train_qui2_fls[col])
    r_list.append(list(result))
  else:
    result = ttest_ind(train_qui2_tr[col], train_qui2_fls[col], equal_var=False)
    r_list.append(list(result))

df = pd.DataFrame(r_list, index = cols, columns = ['통계값', 'P_value'])
df['둘이_다른가'] = df['P_value'].apply(lambda x: '다르다' if x < 0.05 else '같다')
df

,통계값,P_value,둘이_다른가
quality_1,-31.549742,2.660366e-218,다르다
quality_2,-2.377491,1.743125e-02,다르다
quality_5,14.463984,2.075074e-47,다르다
quality_6,-6.544800,5.961198e-11,다르다
quality_7,-7.218764,5.250907e-13,다르다
quality_8,0.494952,6.206340e-01,같다
quality_9,-4.936972,7.937170e-07,다르다
quality_10,23.762820,9.036843e-125,다르다
quality_11,-55.539807,0.000000e+00,다르다
quality_12,-5.273121,1.341602e-07,다르다


In [ ]:
train_qui3 = quality_categorize(train_qui2)
test_qui3 = quality_categorize(test_qui2)

train_qui3['problems'] = train_qui3.user_id.isin(prob_list)
train_qui3_tr = train_qui3[train_qui3['problems']]
train_qui3_fls = train_qui3[train_qui3['problems'] != True]

# T 검정을 하기 위해서 두 표본이 등분산인지 확인할 필요가 있음
# 따라서 등분산 검정을 진행
cols = list(train_qui3.iloc[:,1:-2].columns)

r_list = []

for col in cols:
  result = stats.levene(train_qui3_tr[col].dropna(), train_qui3_fls[col].dropna())
  r_list.append(list(result))
#print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))

df = pd.DataFrame(r_list, index = cols, columns = ['통계값', 'P_value'])
df['등분산성'] = df['P_value'].apply(lambda x: '불만족' if x < 0.05 else '만족')
df

,통계값,P_value,등분산성
quality_1,2717.945783,0.000000e+00,불만족
quality_2,3123.124718,0.000000e+00,불만족
quality_5,31347.129682,0.000000e+00,불만족
quality_6,2606.656698,0.000000e+00,불만족
quality_7,220.687432,6.499446e-50,불만족
quality_8,819.935395,3.062387e-180,불만족
quality_9,501.722687,4.327639e-111,불만족
quality_10,17489.210820,0.000000e+00,불만족
quality_11,2452.525667,0.000000e+00,불만족
quality_12,81.884550,1.445719e-19,불만족


In [ ]:
# 카테고리화 한 경우는 전부 불만족이다.
# ttest_ind에 equal_var=False 을 넣어서 등분산 불만족을 표시해주어야 함.

r_list = []

for col in cols:
  result = ttest_ind(train_qui3_tr[col].dropna(), train_qui3_fls[col].dropna(), equal_var=False)
  r_list.append(list(result))

df = pd.DataFrame(r_list, index = cols, columns = ['통계값', 'P_value'])
df['둘이_다른가'] = df['P_value'].apply(lambda x: '다르다' if x < 0.05 else '같다')
df

,통계값,P_value,둘이_다른가
quality_1,-46.732296,0.000000e+00,다르다
quality_2,-52.324643,0.000000e+00,다르다
quality_5,150.575219,0.000000e+00,다르다
quality_6,-45.514973,0.000000e+00,다르다
quality_7,14.749354,3.153937e-49,다르다
quality_8,28.379437,4.475413e-177,다르다
quality_9,-22.539983,1.820894e-112,다르다
quality_10,201.287485,0.000000e+00,다르다
quality_11,-49.995806,0.000000e+00,다르다
quality_12,-9.059933,1.308068e-19,다르다


In [ ]:
train_qui3_tr

,user_id,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,time,problems
120,10004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2020-11-02 23:20:00,True
121,10004,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2020-11-02 23:20:00,True
122,10004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2020-11-02 23:20:00,True
123,10004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2020-11-02 23:20:00,True
124,10004,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2020-11-02 23:20:00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,24997,0.0,0.0,1.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00,True
828620,24997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00,True
828621,24997,0.0,0.0,2.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00,True
828622,24997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00,True
